In [318]:
# Data Handling
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import os
from numpy.random import seed

# Visualization
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

#PCA
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA


# Load specific forecasting tools 
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback, History, EarlyStopping

from keras.models import Sequential, Model , load_model
from keras.layers import Dense, LSTM, GRU, Flatten, BatchNormalization, Activation, Dropout, TimeDistributed
from keras.layers.convolutional import Conv1D, MaxPooling1D,Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, Callback, History, EarlyStopping
from keras import optimizers, regularizers

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error

Using TensorFlow backend.


In [62]:
!ls ../App/Predictions

CNN2D_pred_dumps.npy  C_LSTM_test_dumps.npy LSTM_pred_dumps.npy
CNN2D_test_dumps.npy  GRU_pred_dumps.npy    LSTM_test_dumps.npy
C_LSTM_pred_dumps.npy GRU_test_dumps.npy    dates_dump.npy


# Data Split investigation

In [55]:
data=pd.read_csv('../App/Data/Cleaned_data_lloyds.csv')

In [56]:
data_train, data_val = train_test_split(data, train_size=0.90, test_size=0.10, shuffle=False)
print('Shape of Training data: ',data_train.shape)

Shape of Training data:  (2588, 32)


In [57]:
print('Shape of Training data: ',data_val.shape)

Shape of Training data:  (288, 32)


In [58]:
val_count=int(data_val.shape[0]*0.8)
test_count=data_val.shape[0]-val_count
print(val_count)
print(test_count)

230
58


In [59]:
data_test=data_val[val_count:][:]
data_val=data_val[:val_count][:]

In [60]:
data_test.tail(2)

,Date,Lloyds_Open,Lloyds_High,Lloyds_Low,Lloyds_Close,Volume,SMA_10,WMA_10,rsi,stoc_k,...,Oil_High,Oil_Low,Oil_Volume,FTSE_Price,FTSE_Open,FTSE_High,FTSE_Low,Weekday,Month,Year
2874,2020-08-26,28.18,28.560,28.03,28.475,91481211.0,28.511,28.397364,48.050768,2.230276,...,46.10,45.54,132510.0,6045.60,6037.01,6050.81,5992.22,2,8,2020
2875,2020-08-27,28.32,28.595,28.08,28.200,94257788.0,28.413,28.340818,44.811213,1.872695,...,45.87,44.56,102260.0,5999.99,6045.60,6062.52,5999.99,3,8,2020


In [61]:
data_val.tail(2)

,Date,Lloyds_Open,Lloyds_High,Lloyds_Low,Lloyds_Close,Volume,SMA_10,WMA_10,rsi,stoc_k,...,Oil_High,Oil_Low,Oil_Volume,FTSE_Price,FTSE_Open,FTSE_High,FTSE_Low,Weekday,Month,Year
2816,2020-06-05,34.51,35.96,34.21,35.545,457376990.0,31.724,32.686364,71.332269,79.479797,...,42.48,39.72,250310.0,6484.30,6341.44,6489.99,6341.44,4,6,2020
2817,2020-06-08,35.20,38.12,35.00,36.880,513534214.0,32.600,33.623818,74.897191,86.646835,...,43.41,40.68,227880.0,6472.59,6484.30,6511.84,6423.06,0,6,2020


# Training data

In [66]:
# load files
lstm_predictions=np.load('../App/Predictions/LSTM_pred_dumps.npy')
lstm_test=np.load('../App/Predictions/LSTM_test_dumps.npy')
dates_dump=np.load('../App/Predictions/dates_dump.npy')
print(len(lstm_predictions))
print(len(lstm_test))
print(len(dates_dump))

40
40
58


In [67]:
dates_dump

array(['2020-06-09', '2020-06-10', '2020-06-11', '2020-06-12',
       '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18',
       '2020-06-19', '2020-06-22', '2020-06-23', '2020-06-24',
       '2020-06-25', '2020-06-26', '2020-06-29', '2020-06-30',
       '2020-07-01', '2020-07-02', '2020-07-03', '2020-07-06',
       '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10',
       '2020-07-13', '2020-07-14', '2020-07-15', '2020-07-16',
       '2020-07-17', '2020-07-20', '2020-07-21', '2020-07-22',
       '2020-07-23', '2020-07-24', '2020-07-27', '2020-07-28',
       '2020-07-29', '2020-07-30', '2020-07-31', '2020-08-03',
       '2020-08-04', '2020-08-05', '2020-08-06', '2020-08-07',
       '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13',
       '2020-08-14', '2020-08-17', '2020-08-18', '2020-08-19',
       '2020-08-20', '2020-08-21', '2020-08-24', '2020-08-25',
       '2020-08-26', '2020-08-27'], dtype=object)

In [68]:
lstm_test

array([32.745, 31.61 , 31.62 , 31.085, 31.255, 31.175, 30.7  , 31.85 ,
       31.025, 31.205, 30.74 , 30.72 , 29.735, 30.455, 30.64 , 30.105,
       30.385, 30.53 , 30.565, 30.41 , 30.505, 30.32 , 30.165, 29.695,
       28.9  , 29.22 , 28.365, 26.205, 26.285, 27.87 , 28.155, 28.175,
       28.16 , 27.785, 28.415, 29.455, 29.69 , 29.18 , 28.95 , 28.625])

In [69]:
lstm_predictions

array([[35.798782],
       [35.99881 ],
       [35.34692 ],
       [34.854332],
       [35.303   ],
       [35.48644 ],
       [35.45772 ],
       [34.62003 ],
       [35.891003],
       [34.9252  ],
       [35.198463],
       [34.35942 ],
       [34.652023],
       [33.225594],
       [34.61349 ],
       [34.506702],
       [34.48201 ],
       [34.472683],
       [34.789932],
       [34.39409 ],
       [34.98081 ],
       [34.98548 ],
       [34.14599 ],
       [34.314507],
       [32.630207],
       [32.34559 ],
       [33.371532],
       [32.353844],
       [29.827759],
       [30.118423],
       [31.649395],
       [31.99806 ],
       [32.527683],
       [32.035847],
       [31.726692],
       [32.41353 ],
       [33.561123],
       [33.14961 ],
       [33.091732],
       [32.405014]], dtype=float32)

# Automation to set Stocker Data 

In [228]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
from pandas.tseries.offsets import BDay

In [226]:
today = pd.datetime.today()
print(today)
print(today + BDay(2))

2020-08-30 17:36:24.542447
2020-09-01 17:36:24.542447


In [218]:
datetime.datetime.now().strftime('%Y-%m-%d')

'2020-08-30'

In [232]:
params = {
  'access_key': '8231f84ca3b18528015548706c8bd76f',
  #  'exchange':'XLON',
    'symbols':'LLOY.XLON'

}
api_result=requests.get('http://api.marketstack.com/v1/eod/latest', params)
date_str=api_result.json()['data'][0]['date']

In [235]:
date_str[:10]

'2020-08-28'

In [215]:
params = {
  'access_key': 'd55d57ff0d2c79990689e03ea82691d8',
#  'from':'USD',
#  'to':'GBP',
  'date':'2020-08-27'
}
api_result=requests.get('http://api.currencylayer.com/historical', params)
api_result.json()

{'success': True,
 'terms': 'https://currencylayer.com/terms',
 'privacy': 'https://currencylayer.com/privacy',
 'historical': True,
 'date': '2020-08-27',
 'timestamp': 1598572799,
 'source': 'USD',
 'quotes': {'USDAED': 3.673195,
  'USDAFN': 77.01432,
  'USDALL': 105.17487,
  'USDAMD': 487.030492,
  'USDANG': 1.795391,
  'USDAOA': 593.337056,
  'USDARS': 73.877504,
  'USDAUD': 1.375648,
  'USDAWG': 1.8,
  'USDAZN': 1.701015,
  'USDBAM': 1.656992,
  'USDBBD': 2.019554,
  'USDBDT': 84.80976,
  'USDBGN': 1.65477,
  'USDBHD': 0.376979,
  'USDBIF': 1930.1975,
  'USDBMD': 1,
  'USDBND': 1.366652,
  'USDBOB': 6.905967,
  'USDBRL': 5.570198,
  'USDBSD': 1.000216,
  'USDBTC': 8.8391383e-05,
  'USDBTN': 73.79032,
  'USDBWP': 11.563148,
  'USDBYN': 2.662566,
  'USDBYR': 19600,
  'USDBZD': 2.016038,
  'USDCAD': 1.312245,
  'USDCDF': 1949.999981,
  'USDCHF': 0.9092,
  'USDCLF': 0.028427,
  'USDCLP': 784.399135,
  'USDCNY': 6.890203,
  'USDCOP': 3816.43,
  'USDCRC': 595.61128,
  'USDCUC': 1,
  'US

In [255]:
model_data=pd.read_csv('model_data.csv')
model_data.tail()

,Date,Lloyds_Close
2871,21/08/2020,28.100
2872,24/08/2020,28.600
2873,25/08/2020,28.200
2874,26/08/2020,28.475
2875,27/08/2020,28.200


In [256]:
record_dict={'Date':'2009-03-04',
             'Lloyds_Close':'55.6'}
model_data=model_data.append(record_dict,ignore_index=True)
model_data.tail()

,Date,Lloyds_Close
2872,24/08/2020,28.6
2873,25/08/2020,28.2
2874,26/08/2020,28.475
2875,27/08/2020,28.2
2876,2009-03-04,55.6


In [257]:
a='2020-02-04'
a.split('-')

['2020', '02', '04']

In [258]:
import pandas_market_calendars as mcal

In [262]:
LSE = mcal.get_calendar('LSE')
start_date_str='2020-08-28'
a=LSE.valid_days(start_date=start_date_str, end_date='2020-12-31')[:20]
next_day=a[1].strftime('%d/%m/%Y')

In [263]:
next_day

'01/09/2020'

In [321]:
data=pd.read_csv('model_data.csv')

In [322]:
data.tail()

,Date,Lloyds_Close
2873,25/08/2020,28.2
2874,26/08/2020,28.475
2875,27/08/2020,28.2
2876,28/08/2020,28.35
2877,01/09/2020,


In [316]:
data[-2:-1][:]['Date'].values[0]

'28/08/2020'

In [308]:
data=data[:-1][:]['Date']

In [309]:
data

0       03/03/2009
1       04/03/2009
2       05/03/2009
3       06/03/2009
4       09/03/2009
           ...    
2872    24/08/2020
2873    25/08/2020
2874    26/08/2020
2875    27/08/2020
2876    28/08/2020
Name: Date, Length: 2877, dtype: object

In [284]:
data.dtypes

Date            object
Lloyds_Close    object
dtype: object

In [285]:
data.head(5)

,Date,Lloyds_Close
0,03/03/2009,45.5
1,04/03/2009,47.7
2,05/03/2009,39.32
3,06/03/2009,42.0
4,09/03/2009,43.7


In [286]:
data['Date']=data['Date'].apply(lambda x: x.split('/')[2]+'-'+x.split('/')[1]+'-'+x.split('/')[0])

In [298]:
data.tail(5)

,Date,Lloyds_Close
2872,2020-08-24,28.6
2873,2020-08-25,28.2
2874,2020-08-26,28.475
2875,2020-08-27,28.2
2876,2020-08-28,28.35


In [288]:
data['Date']=pd.to_datetime(data['Date'])

In [297]:
data[-3:][:]

,Date,Lloyds_Close
2874,2020-08-26,28.475
2875,2020-08-27,28.2
2876,2020-08-28,28.35


In [319]:
preds=np.load('predictions_dump.npy')

In [320]:
preds

array([29.00961373, 29.03863194, 29.02498553, 29.01431446, 29.00987747,
       29.0052206 , 28.99966014, 28.99410113, 28.98867214, 28.98324726,
       28.97780378])

In [323]:
dates=data.tail(11)['Date'].values

In [324]:
dates

array(['17/08/2020', '18/08/2020', '19/08/2020', '20/08/2020',
       '21/08/2020', '24/08/2020', '25/08/2020', '26/08/2020',
       '27/08/2020', '28/08/2020', '01/09/2020'], dtype=object)